In [1]:
import geopandas as gpd
from keplergl import KeplerGl
import numpy as np
from shapely.geometry import Point, LineString, shape

### Data

In [2]:
path = './tundraSwan_USGS_ASC_argos_diag_filteredLocations.csv' #DATA FROM USGS

df_swan = gpd.read_file(path)
df_swan = df_swan.query("Location_DAF_Filter == '0'")           #FILTER BY QUALITY LEVEL 
df_swan

In [30]:
df_swan['year'] = df_swan['Location_Timestamp_UTC'].apply(lambda x : x[:4]).astype(int)  #RETRIEVE THE YERAR
df_swan_2011 = df_swan.loc[df_swan['year'] == 2011]
df_swan_2011

,Data_Release_DOI,Data_Release_Version,Animal_ID,Animal_Species,Location_Timestamp_UTC,Latitude,Longitude,Location_DAF_Filter,Location_Class,Location_NOPC,...,Lat_ShedDead,Lon_ShedDead,Pass_Satellite_ID,Pass_Messages_N,Pass_Messages_gt120dB,Pass_Messages_BestLeveldB,Pass_Duration,Pass_EstimatedFrequency,geometry,year
1902,10.5066/P9KBR79C,v01,TUSW_KS7,Cygnus columbianus,2011/02/12 05:52:40,42.62,-121.947,0,1,1,...,,,MA,5,0,-126,452,401675350.880,None,2011
1903,10.5066/P9KBR79C,v01,TUSW_KS7,Cygnus columbianus,2011/02/12 10:25:30,42.625,-122.035,0,A,3,...,,,NN,3,0,-130,390,401675334.010,None,2011
1904,10.5066/P9KBR79C,v01,TUSW_KS7,Cygnus columbianus,2011/02/16 10:43:17,42.873,-121.738,0,A,1,...,,,NP,3,0,-135,324,401675287.600,None,2011
1905,10.5066/P9KBR79C,v01,TUSW_KS7,Cygnus columbianus,2011/02/16 11:24:54,42.879,-121.726,0,A,3,...,,,NN,3,0,-136,259,401675294.360,None,2011
1906,10.5066/P9KBR79C,v01,TUSW_KS7,Cygnus columbianus,2011/02/17 10:33:49,42.854,-121.76,0,B,2,...,,,NP,2,0,-135,64,401675294.360,None,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96043,10.5066/P9KBR79C,v01,TUSW_NS3,Cygnus columbianus,2011/01/13 02:51:42,38.188,-76.917,0,B,1,...,,,MA,2,0,-129,71,401672275.870,None,2011
96045,10.5066/P9KBR79C,v01,TUSW_NS3,Cygnus columbianus,2011/01/28 07:16:34,38.104,-76.997,0,2,1,...,,,NP,6,0,-133,366,401672238.730,None,2011
96046,10.5066/P9KBR79C,v01,TUSW_NS3,Cygnus columbianus,2011/01/28 09:15:14,38.152,-76.877,0,B,1,...,,,NK,2,0,-136,74,401672275.870,None,2011
96047,10.5066/P9KBR79C,v01,TUSW_NS3,Cygnus columbianus,2011/02/01 10:57:38,38.102,-77.023,0,A,1,...,,,NK,3,0,-134,303,401672203.170,None,2011


### Create one DataFrame per Swan

In [31]:
our_swans = df_swan_2011['Animal_ID'].unique()
dict_df_swan = dict()
for swan in our_swans:
    dict_df_swan[swan] = df_swan_2011.query("Animal_ID == '{}'".format(swan))

In [32]:
our_swans

array(['TUSW_KS7', 'TUSW_CB3', 'TUSW_KS6', 'TUSW_KZ1', 'TUSW_CB6',
       'TUSW_NS2', 'TUSW_KZ8', 'TUSW_KZ10', 'TUSW_NS5', 'TUSW_NS4',
       'TUSW_YK10', 'TUSW_YK2', 'TUSW_KS8', 'TUSW_CB4', 'TUSW_YK3',
       'TUSW_CB1', 'TUSW_KS9', 'TUSW_CB10', 'TUSW_NS10', 'TUSW_CB9',
       'TUSW_CB7', 'TUSW_KZ4', 'TUSW_NS8', 'TUSW_NS7', 'TUSW_KS4',
       'TUSW_KS3', 'TUSW_KS2', 'TUSW_CB2', 'TUSW_CB5', 'TUSW_NS1',
       'TUSW_NS9', 'TUSW_KZ7', 'TUSW_KZ9', 'TUSW_YK4', 'TUSW_NS3'],
      dtype=object)

In [33]:
dict_df_swan_line = dict()
for swan in dict_df_swan.keys():
    print(swan)
    dict_df_swan[swan].replace("", float("NaN"), inplace = True)
    dict_df_swan[swan].dropna(subset =['Latitude', 'Longitude'], inplace = True)
    dict_df_swan[swan]['Latitude']    = dict_df_swan[swan]['Latitude'].astype(float)
    dict_df_swan[swan]['Longitude']   = dict_df_swan[swan]['Longitude'].astype(float)
    dict_df_swan[swan]['geometry']    = gpd.points_from_xy(dict_df_swan[swan].Longitude, dict_df_swan[swan].Latitude)
    dict_df_swan[swan]['geometry']    = LineString(dict_df_swan[swan]['geometry'].tolist())

TUSW_KS7
TUSW_CB3
TUSW_KS6
TUSW_KZ1
TUSW_CB6
TUSW_NS2
TUSW_KZ8
TUSW_KZ10
TUSW_NS5
TUSW_NS4
TUSW_YK10
TUSW_YK2
TUSW_KS8


C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\pandas\core\frame.py:4385: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

TUSW_CB4
TUSW_YK3
TUSW_CB1
TUSW_KS9
TUSW_CB10
TUSW_NS10
TUSW_CB9
TUSW_CB7
TUSW_KZ4
TUSW_NS8
TUSW_NS7
TUSW_KS4
TUSW_KS3
TUSW_KS2
TUSW_CB2
TUSW_CB5
TUSW_NS1
TUSW_NS9
TUSW_KZ7
TUSW_KZ9
TUSW_YK4
TUSW_NS3


C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [35]:
map_tio = KeplerGl(height=500, data=dict_df_swan)
map_tio.save_to_html(file_name="E:\\Pro\\BDD\\ARGOS\\Lac_des_Cygnes.html")

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to E:\Pro\BDD\ARGOS\Lac_des_Cygnes.html!
